# Redes Neuronales - Trabajo Práctico N° 1 - Ejercicio 2 - Notebook #2
En esta segunda notebook, se busca definir cuál métrica es más apropiada para analizar la performance del modelo y qué hiper parámetros se van a utilizar para el ajuste del modelo acorde a la validación. Finalmente, estas decisiones se vuelcan en la selección del mejor modelo para el problema de la clasificación de correos electrónicos asociados grupos de noticias.

### Fuentes útiles
* https://en.wikipedia.org/wiki/Bessel%27s_correction
* https://en.wikipedia.org/wiki/Kernel_density_estimation
* https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
* https://stackoverflow.com/questions/58046129/can-someone-give-a-good-math-stats-explanation-as-to-what-the-parameter-var-smoo
* https://scikit-learn.org/stable/modules/density.html

### Integrantes del grupo
* Gaytan, Joaquín Oscar
* Kammann, Lucas Agustín

# 1. Métrica
La métrica a utilizar para cuantificar la performance de los modelos, seleccionar los hiperparámetros y validarlos, será la **sensibilidad** o **recall**.

## 1.1. Justificación
Se emplea el recall o sensibilidad respecto de los positivos, que se calcula como:
$$recall = \frac{TP}{TP+FN}$$
Es decir, esta métrica da información sobre la proporción de positivos identificados sobre el total de positivos (reales). En el caso del diagnóstico de una enfermedad, nos interesa minimizar el número de falsos negativos, dado que una persona clasificada como negativo pero que efectivamente esté enferma puede no recibir el tratamiento correspondiente, empeorando su cuadro y poniendo 

# 2. Preparación de los datasets

## 2.1. Cargando el dataset original

In [275]:
import pandas as pd
import numpy as np

In [276]:
# Read database from .csv
df = pd.read_csv('../assets/diabetes.csv', delimiter=',')

## 2.2. Sustitución de valores nulos
Los valores nulos de las variables o características para la clasificación se reemplazan por NaN o Not a Number, para evitar que sean procesados en el análisis estadístico posterior, de esta forma luego serán reemplazados por algún estadístico. Se los remueve porque son valores inválidos acorde a la interpretación física de las variables diagnóstico con las cuales se realiza la clasificación y/o predicción.

In [277]:
# Filtering Glucose values
df['Glucose'].replace(0, np.nan, inplace=True)

# Filtering Blood Pressure values
df['BloodPressure'].replace(0, np.nan, inplace=True)

# Filtering Skin Thickness values
df['SkinThickness'].replace(0, np.nan, inplace=True)

# Filtering Insulin values
df['Insulin'].replace(0, np.nan, inplace=True)

# Filtering Body Mass Index values
df['BMI'].replace(0, np.nan, inplace=True)

## 2.3. Filtrado de outliers

Inicialmente se dejaron los outliers, pero se evidenció que en algunas iteraciones se producía una caída en la métrica,y bajo la suposición de que los outliers estaban afectando fuertemente a la estimación de los parámetros de los modelos de probabilidad, se los removió y como consecuencia mejoró la performance del modelo. En conclusión, es necesario eliminar estos valores porque son poco representativos de la población y afectan directamente a los modelos de probabilidad empleados.

In [278]:
from src.helper import remove_outliers

In [279]:
for column in df.columns:
    remove_outliers(df, column)

## 2.4. Separación de datasets
Se separa el dataset original en los datasets de train, valid y test. Además, se debe corregir que los valores inválidos del dataset original fueron reemplazados por el valor NaN.

In [280]:
from sklearn.model_selection import train_test_split

In [281]:
# Splitting into the total train and the test datasets, because
# the total train contains the train and valid datasets used for
# hiper parameter selection
train, test = train_test_split(df, test_size=0.3, random_state=44)

## 2.5. Sustitución de valores inválidos
Se filtran los valores inválidos de cada una de las variables, y se los reemplaza utilizando la media obtenida en el conjunto de entrenamiento. Particularmente, se opta por emplear la media de todo el conjunto de entrenamiento, para no introducir sesgo esencialmente dentro del conjunto empleado para la evaluación del modelo. Es necesario hacer esto para conseguir homogeneizar las muestras de cada varaibles de diagnóstico.

In [282]:
# Compute the mean of training
train_means = train.mean().to_numpy()

# Replacing nan values of the test dataset with training mean values
for index, column in enumerate(train.columns):
    train.loc[:,column].replace(np.nan, train_means[index], inplace=True)

# Replacing nan values of the test dataset with training mean values
for index, column in enumerate(test.columns):
    test.loc[:,column].replace(np.nan, train_means[index], inplace=True)

C:\anaconda\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


## 2.6. Conjuntos de entrenamiento y evaluación
Se construyen los conjuntos de entrenamiento y evaluación, y además se filtran aquellas variables de interés.

### Remoción de **DiabetesPedigreeFunction** 
Se elimina la variable **DiabetesPedigreeFunction** dado que el análisis estadístico previamente realizado dejaba entrever que la información que aporta es baja. Esto lo indicaron dos factores, en primer lugar el poco cambio entre las distribuciones condicionadas, y en segundo lugar la baja correlación con la variable de salida.

### Remoción de **BloodPressure**
Se elimina la variable **BloodPressure** dado que el análisis estadístico previamente realizado mostró que la información que aporta es baja. Esto lo indicador dos factores, en primer lugar el poco cambio entre las distribuciones condicionadas, y en segundo lugar, la baja correlación con la variable de salida.

In [283]:
# Global filter of the variables
global_variables_filter = np.array([True, True, False, True, True, True, False, True, False])
global_variables_count = global_variables_filter.sum()

In [284]:
# Extracting the inputs and outputs of the train dataset
x_train = train.to_numpy()[:,global_variables_filter]
y_train = train.to_numpy()[:,8]

# Extracting the inputs and outputs of the test dataset
x_test = test.to_numpy()[:,global_variables_filter]
y_test = test.to_numpy()[:,8]

In [285]:
train.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000
mean,3.670412,122.132959,72.143713,28.953003,131.703297,32.373372,0.421482,32.758491,0.331471
std,3.229410,30.544791,10.935330,8.451481,52.528972,6.382944,0.242506,11.179596,0.471181
min,0.000000,44.000000,40.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,100.000000,64.000000,25.000000,116.000000,27.600000,0.238000,24.000000,0.000000
50%,3.000000,118.000000,72.143713,28.953003,131.703297,32.373372,0.361000,29.000000,0.000000
75%,6.000000,141.000000,80.000000,32.000000,131.703297,36.600000,0.551000,40.000000,1.000000
max,13.000000,199.000000,104.000000,56.000000,342.000000,50.000000,1.191000,66.000000,1.000000


In [286]:
test.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000,231.000000
mean,4.054850,120.659160,72.055707,28.837352,133.156891,31.824214,0.448196,32.911329,0.389610
std,3.354694,30.222273,10.753877,7.795778,49.215393,6.471815,0.249987,10.758273,0.488721
min,0.000000,56.000000,44.000000,7.000000,18.000000,18.200000,0.084000,21.000000,0.000000
25%,1.000000,99.000000,64.500000,25.500000,131.703297,27.250000,0.251500,24.000000,0.000000
50%,3.000000,115.000000,72.000000,28.953003,131.703297,32.000000,0.389000,30.000000,0.000000
75%,6.500000,138.500000,80.000000,32.000000,131.703297,35.750000,0.646000,40.000000,1.000000
max,13.000000,198.000000,104.000000,48.000000,360.000000,49.600000,1.144000,65.000000,1.000000


# 3. Selección, validación y evaluación de modelos
Para poder realizar la validación del modelo y escoger aquellos hiper parámetros que obtienen la mejor performace según la métrica, se utiliza el método de k-folding, dado que se cuenta con una cantidad de datos pequeña. Si no se decidiera utilizar k-folding, al tener un conjunto de validación tan pequeño o reducido, la varianza en el estimador de la métrica es demasiado grande y la estimación posee mucho ruido, lo cual es un problema porque provocaría elegir un modelo equivocado.

Este algoritmo está implementado dentro de la función **GridSearchCV**, que además busca optimizar la métrica elegida (recall) variando los hiper parámetros del modelo.

Finalmente, se entrena al modelo con los hiper parámetros que resultan de este análisis.

In [287]:
from sklearn.model_selection import GridSearchCV

In [288]:
from sklearn.metrics import recall_score

In [289]:
import itertools

## 3.1. Modelo utilizando Gaussian Naive Bayes sin hiper parámetros
En este punto, se busca entrenar el model y evaluar su performance, contrastando los resultados obtenidos por la implementación propia con la implementación de sklearn

### 3.1.1. Versión del clasificador propia

In [290]:
from src.gaussian_naive_bayes import BinaryGaussianNaiveBayes

In [291]:
# Train the found model with the complete train set
classifier = BinaryGaussianNaiveBayes()
classifier.fit(x_train, y_train)

In [292]:
# Predictions using the test dataset and computing the score
predictions = classifier.predict(x_test)
score = recall_score(y_test, predictions)

In [293]:
print(f'Score of the model {np.round(score, 3)}')

Score of the model 0.644


### 3.1.2. Versión del clasificador sklearn

In [294]:
from sklearn.naive_bayes import GaussianNB

# Create and train the model
c = GaussianNB()
c.fit(x_train, y_train)

# Predict and compute score
p = c.predict(x_test)
s = recall_score(y_test, p)

# Show the resulting score
print(f'Score of the model {np.round(s, 3)}')

Score of the model 0.644


## 3.2. Modelo utilizando Gaussian Naive Bayes con hiper parámetros

### Smoothing
En las variables aleatorias continuas con distribución normal o gaussiana, cuando es estiman sus parámetros mediante un conjunto de entrenamiento, se corre el riesgo de **underfitting** y **overfitting**, es decir, que una mala estimación de la distribución de probabilidad puede excluir valores de su rango que luego hacen que el modelo no pueda predecir con éxito. Entonces, una técnica empleada para solucionar este tipo de problemas, consiste en agregar un suavizado en las variables gaussianas, sumando un término al desvío estándar.

### Corrección de Bessel
En la estadística, los estadísticos se definen como funciones que se aplican sobre muestras y datos, y un caso particular de ellos son los estimadores que se emplean para estimar parámetros poblacionales como la media o el desvío estándar en este caso. Una característica buscada de los estimadores, es que sean sin sesgo, para ello el estimador de la varianza debe ser,

$$ s^{2} = \frac{1}{n-1} \cdot \sum_{i=1}^{n}(x_i - x_{mean})^{2}$$

Y, por lo general, a veces se calcula

$$ s^{2} = \frac{1}{n} \cdot \sum_{i=1}^{n}(x_i - x_{mean})^{2}$$

Entonces, la **corrección de Bessel** es un factor multiplicativo que corrige el término del denominador.

### Filtro de variables
El filtro de variables consiste en agregar un hiper parámetro que le permite al modelo escoger qué variables usar, y de esta forma, encontrar aquella combinación con la cual la predicción se optimiza. En principio, se sabe que algunas variables guardan una correlación medianamente fuerte, y que otras no ven su distribución muy afectada por la clase a la que pertenezca el individuo, por ello, resulta razonable que existan modelos que no empleen la totalidad de las variables o características que se proveen en la base de datos.

In [295]:
from src.gaussian_naive_bayes import BinaryGaussianNaiveBayes

In [ ]:
%%time

# Declaring the hiper parameters and their values for the GridSearchCV
# to search the best model according to the performance
parameters = {
    'std_smoothing': [0, 0.1, 1],
    'std_correction': [False, True],
    'filter_variables': list(itertools.product([True, False], repeat=global_variables_count)),
}

# Estimator or model
estimator = BinaryGaussianNaiveBayes()

# Search the best model
grid = GridSearchCV(estimator, parameters, cv=10, scoring='recall', n_jobs=-1)
grid.fit(x_train, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
# Train the found model with the complete train set
classifier = BinaryGaussianNaiveBayes(
    std_smoothing=grid.best_params_['std_smoothing'], 
    std_correction=grid.best_params_['std_correction'], 
    filter_variables=grid.best_params_['filter_variables']
)
classifier.fit(x_train, y_train)

In [ ]:
# Predictions using the test dataset and computing the score
predictions = classifier.predict(x_test)
score = recall_score(y_test, predictions)

In [ ]:
print(f'Score of the model {np.round(score, 3)}')

## 3.3. Modelo Naive Bayes con distribución por variable

A partir del análisis estadístico realizado sobre las variables se sacó la conclusión de que en la realidad algunas variables o características físicas del problema utilizadas para la clasificación poseen una distribución que se aparta bastante del modelo probabilístico de una variable gaussiana. Entonces, en vista de esta observación, se desea poder replicar el concepto del clasificador **Gaussian Naive Bayes**, pero dando la libertad de que cada variable trabaje con una distribución propia.

En primera instancia, se implementan las distribuciones **gaussiana** y **exponencial**.

*Nota: Se excluyeron los hiper parámetros que pueden estar más relacionados a una distribución que otra, principalmente para no complejizar la búsqueda del modelo óptimo y probar en una primera instancia diferentes distribuciones para cada variable.*

In [ ]:
from src.mixed_naive_bayes import BinaryMixedNaiveBayes

In [ ]:
%%time

# Declaring the hiper parameters and their values for the GridSearchCV
# to search the best model according to the performance
parameters = {
    'filter_variables': list(itertools.product([True], repeat=8)),
    'variables_models': [
        # Pregnancies | Glucose | BloodPressure | SkinThickness | Insulin | BMI | DiabetesPedigreeFunction | Age
        [ 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian'],
        [ 'exponential', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian', 'exponential', 'exponential'],
        [ 'exponential', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian', 'exponential', 'gaussian'],
        [ 'exponential', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian', 'gaussian', 'exponential'],
        [ 'exponential', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian', 'gaussian', 'gaussian'],
        [ 'exponential', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'exponential'],
        [ 'exponential', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian'],
        [ 'exponential', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential'],
        [ 'exponential', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian'],
        [ 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian', 'exponential', 'exponential'],
        [ 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian', 'exponential', 'gaussian'],
        [ 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian', 'gaussian', 'exponential'],
        [ 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian', 'gaussian', 'gaussian'],
        [ 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'exponential'],
        [ 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential', 'gaussian'],
        [ 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'exponential'],
    ]
}

# Estimator or model
estimator = BinaryMixedNaiveBayes()

# Search the best model
grid = GridSearchCV(estimator, parameters, cv=10, scoring='recall', n_jobs=-1)
grid.fit(x_train, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
# Train the found model with the complete train set
classifier = BinaryMixedNaiveBayes(
    variables_models=grid.best_params_['variables_models'], 
    filter_variables=grid.best_params_['filter_variables']
)
classifier.fit(x_train, y_train)

In [ ]:
# Predictions using the test dataset and computing the score
predictions = classifier.predict(x_test)
score = recall_score(y_test, predictions)

In [ ]:
print(f'Score of the model {np.round(score, 3)}')

## 3.4. Modelo utilizando Kernel Density Estimator
A fin de comparar el desempeño del clasificador gaussiano, se propone el uso de un clasificador con KDE (Kernel Density Estimation). Este es un método para estimar funciones de densidad de probabilidad que se basa en la información de las muestras en un entorno de valor determinado. La estimación se define como:

$$\hat{f}(x) = \frac{1}{nh} \sum_{i=1}^n{K\left(\frac{x-x_i}{h}\right)}$$

Donde $K(x)$ es una función no negativa denominada **kernel** y $n$ representa el número de observaciones. Además, $h$ es un parámetro positivo llamado **bandwidth** que se vincula con la cantidad de puntos a tener en cuenta para la estimación de un valor puntual de $\hat{f}$. Para el caso particular de este clasificador se emplean dos tipos de funciones kernel: ‘gaussian’, ‘tophat’, ‘epanechnikov’, ‘exponential’, ‘linear’, ‘cosine’.

In [ ]:
from src.kde_naive_bayes import BinaryKDENaiveBayes

In [ ]:
%%time

# Declaring the hiper parameters and their values for the GridSearchCV
# to search the best model according to the performance
parameters = {
    'kernel': ['exponential', 'tophat', 'linear', 'cosine', 'gaussian', 'epanechnikov'],
    'bandwidth': [0.01, 0.1, 1, 10],
    'filter_variables': list(itertools.product([True, False], repeat=8)),
}

# Estimator or model
estimator = BinaryKDENaiveBayes()

# Search the best model
grid = GridSearchCV(estimator, parameters, cv=10, scoring='recall', n_jobs=-1)
grid.fit(x_train, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
# Train the found model with the complete train set
classifier = BinaryKDENaiveBayes(
    kernel=grid.best_params_['kernel'], 
    bandwidth=grid.best_params_['bandwidth'], 
    filter_variables=grid.best_params_['filter_variables']
)
classifier.fit(x_train, y_train)

In [ ]:
# Predictions using the test dataset and computing the score
predictions = classifier.predict(x_test)
score = recall_score(y_test, predictions)

In [ ]:
print(f'Score of the model {np.round(score, 3)}')